In [1]:
import os
from iou_constants import data_dir, dimension, num_data, temp_export_type, binvox_program_path, npy_types
from tqdm import tqdm
import numpy as np
import trimesh
from multiprocessing import Pool, cpu_count
from trimesh.exchange import binvox
import subprocess

In [2]:
obj_list = os.listdir(data_dir)

In [3]:
def f(idx):
    obj_name = obj_list[idx]
    for npy_type in list(npy_types):
        try:
            voxel_matrix = np.load(os.path.join(data_dir, obj_name, "{}_{}.npy".format(obj_name, npy_type)), allow_pickle=True)
        except:
            print("cannot load {}_{}.npy".format(obj_name, npy_type))
            return obj_name

        try:
            assert(voxel_matrix.shape == (dimension, dimension, dimension))
        except:
            print(obj_name, "has shape", voxel_matrix.shape, "instead of", (dimension, dimension, dimension))
            return obj_name
    return None


with Pool(cpu_count()) as p:
    error_list = p.map(f, list(range(num_data)))
error_list = np.array(error_list)
error_list = error_list[error_list != None]
print("errors list:")
print(list(error_list))

errors list:
[]


In [4]:
print("number of errors:", len(error_list))

number of errors: 0


In [5]:
import trimesh
from trimesh.exchange import binvox
import subprocess
import os
import numpy as np
from iou_constants import data_dir, code_dir, dimension, temp_export_type, binvox_program_path

remove_file_types = ['binvox', 'npy', 'off']

def temp_binvox(temp_export_path):
    output = subprocess.run('export LD_PRELOAD=/share/apps/local/lib/libGL.so; xvfb-run -d -w 0 {} -d {} {}'.format(binvox_program_path, dimension, temp_export_path),
                            shell=True, capture_output=True)
    try:
        assert(output.returncode == 0 and output.stdout.decode('ascii').split('\n')[-3] == 'done')
#         assert(output.returncode == 0)
    except:
        raise Exception(output.stdout.decode('ascii'))
    
def binvox_npy(name_base, binvox_path, align_center=False):
    with open(binvox_path, 'rb') as fp:
        voxel_grid = np.array(binvox.load_binvox(fp, axis_order='xyz').matrix)
    
    if align_center:
        coordinates = np.where(voxel_grid == 1)
        print(coordinates)
        print((voxel_grid == 0).all() == True)
        new_coordinates = [None] * 3 
        for i in range(3):
            min_diff = min(coordinates[i])
            max_diff = dimension - 1 - max(coordinates[i])
            new_coordinates[i] = coordinates[i] + (max_diff - min_diff) // 2
            voxel_grid = np.zeros((dimension, dimension, dimension))
            voxel_grid[new_coordinates[0], new_coordinates[1], new_coordinates[2]] = 1
    
    np.save(name_base.format('npy'), voxel_grid)
    
def mesh_npy(mesh, type_name, obj_name, obj_dir):
    name_base = os.path.join(obj_dir, '{}_{}.{}'.format(obj_name, type_name, '{}'))
    
    npy_path = name_base.format('npy')
    if not os.path.exists(npy_path):
        binvox_path = name_base.format('binvox')
        if not os.path.exists(binvox_path):
            temp_export_path = name_base.format(temp_export_type)
            if not os.path.exists(temp_export_path):
                with open(temp_export_path, 'wb') as fp:
                    mesh.export(fp, file_type=temp_export_type)
            else:
                print("object", obj_name, "already have", type_name, temp_export_type, "file generated.")
            temp_binvox(temp_export_path)
        else:
            print("object", obj_name, "already have", type_name, "binvox file generated.")
        binvox_npy(name_base, binvox_path, align_center=(type_name=='align_center'))
    else:
        print("object", obj_name, "already have npy file generated.")
    
def regenerate_voxel_grid(error_list):
    for obj_name in tqdm(error_list):
        obj_dir = os.path.join(data_dir, obj_name)
        for file_type in remove_file_types:
            os.system("rm {}".format(os.path.join(data_dir, obj_name, "*.{}".format(file_type))))
        files = os.listdir(obj_dir)
        for file_name in files:
            if 'obj' in file_name:
                obj_path = os.path.join(obj_dir,file_name)
                break

#         # align center
#         mesh = trimesh.load_mesh(obj_path)
#         mesh_npy(mesh, 'align_center', obj_name, obj_dir)

        # min-max normalization
        mesh = trimesh.load_mesh(obj_path)
        [bbox_min, bbox_max] = mesh.bounds
        mesh.vertices = (mesh.vertices - bbox_min) / (bbox_max - bbox_min)
        mesh_npy(mesh, 'normalized', obj_name, obj_dir)
        
regenerate_voxel_grid(error_list)

  0%|          | 0/103 [00:00<?, ?it/s]<ipython-input-5-42049aa0565c>:75: RuntimeWarning: invalid value encountered in true_divide
  mesh.vertices = (mesh.vertices - bbox_min) / (bbox_max - bbox_min)
  0%|          | 0/103 [00:00<?, ?it/s]


Exception: 
--- [binvox] mesh voxelizer, version 1.33, build #746 on 2021/03/27 20:51:24, running on Linux
--- written by Patrick Min, 2004-2021

loading model file...
MeshFileIdentifier::*create_mesh_file(/scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalized.off)
OffMeshFile::load
  running [wc -l /scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalized.off]
8681 /scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalized.off
  starting to read 2974 vertices
  Error in /scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalized.off at line 3:
    expected number, but didn't find one
  Error in /scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalized.off at line 3:
    expected number, but didn't find one
  Error in /scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalized.off at line 4:
    expected number, but didn't find one
  Error in /scratch/yk1962/ABC2/data/obj_correct_num/00032461/00032461_normalize